# Kaggle competition: ASHRAE - Great Energy Predictor III

In [1]:
# Thorsten

import os
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from timeit import default_timer as timer

In [4]:
# Check available data files
datafiles = []
for dirname, _, filenames in os.walk('input/'):
    datadir = dirname
    for filename in filenames:
        datafiles.append(filename)

print(datadir)
print(datafiles)
# os.path.join(dirname, filename)

input/
['building_metadata.csv', 'sample_submission.csv', 'test.csv', 'train.csv', 'weather_test.csv', 'weather_train.csv']


In [5]:
# Read data
buildings = pd.read_csv(os.path.join(datadir, 'building_metadata.csv'))
train = pd.read_csv(os.path.join(datadir, 'train.csv'))
test = pd.read_csv(os.path.join(datadir, 'test.csv'))

print(buildings.head())
print(train.head())
print(test.head())

   site_id  building_id primary_use  square_feet  year_built  floor_count
0        0            0   Education         7432      2008.0          NaN
1        0            1   Education         2720      2004.0          NaN
2        0            2   Education         5376      1991.0          NaN
3        0            3   Education        23685      2002.0          NaN
4        0            4   Education       116607      1975.0          NaN
   building_id  meter            timestamp  meter_reading
0            0      0  2016-01-01 00:00:00            0.0
1            1      0  2016-01-01 00:00:00            0.0
2            2      0  2016-01-01 00:00:00            0.0
3            3      0  2016-01-01 00:00:00            0.0
4            4      0  2016-01-01 00:00:00            0.0
   row_id  building_id  meter            timestamp
0       0            0      0  2017-01-01 00:00:00
1       1            1      0  2017-01-01 00:00:00
2       2            2      0  2017-01-01 00:00:00
3   

In [6]:
# Explore dataset
print('Number of readings:', train['building_id'].count())
print('Number of buildings:', train['building_id'].nunique())
print('Average number of meter readings per building:', train.groupby('building_id')['meter_reading'].nunique().mean())

Number of readings: 20216100
Number of buildings: 1449
Average number of meter readings per building: 5604.807453416149


In [7]:
# Average meter reading for all buildings and meter types
averages = train.groupby(['building_id', 'meter'])['meter_reading'].mean().round(4)
averages[10:20]

building_id  meter
9            0          72.7147
             1         664.8260
10           0         980.9485
11           0         296.3925
12           0         143.4260
13           0         228.0089
             1        2007.9555
14           0         259.5876
             1        2521.5515
15           0         164.8777
Name: meter_reading, dtype: float64

In [8]:
# Create own train and test set to estimate model performance
split_train, split_test = train_test_split(train, test_size=0.1)
split_test.head()

,building_id,meter,timestamp,meter_reading
13651548,844,0,2016-09-06 04:00:00,20.100
19446744,133,0,2016-12-18 10:00:00,55.700
3629289,19,0,2016-03-09 11:00:00,0.000
9566520,1003,1,2016-06-25 02:00:00,117.267
16518672,132,0,2016-10-27 00:00:00,64.200


In [40]:
# Find averages for building and meter in test set
prediction = []
for row in split_test.itertuples():
    prediction.append(averages.loc[getattr(row, 'building_id')][getattr(row, 'meter')])
    
prediction[:10]

[21.0633,
 55.863,
 126.3557,
 94.683,
 52.4614,
 158.8238,
 415.0645,
 174.2309,
 485.1112,
 0.4114]

In [ ]:
# Calculate error (Root Mean Squared Logarithmic Error)
meter_reading = split_test['meter_reading']
rmsle = np.sqrt(mean_squared_log_error(meter_reading, prediction))
print(rmsle)